In [1]:
from labjack import ljm
import time
import numpy as np
import visa
import matplotlib.pyplot as plt

import lakeshore_util

%matplotlib notebook

In [2]:
#Labjack
ps_switch = 'FIO0'
v_ctrl = 'DAC0'
v_meas = 'AIN0_EF_READ_A'
v_off = 'AIN0_EF_READ_C'
i_meas = 'AIN1'
rms_set = 'AIN0_EF_INDEX'

labjack = ljm.openS("T7", "usb", "CryoTester_ljT7")
ljm.eWriteName(labjack, ps_switch, 1) #Shutdown the power supply by default
ljm.eWriteName(labjack, rms_set, 10)
#Multimeters
rm = visa.ResourceManager()
multi1 = rm.open_resource('GPIB0::26::INSTR')
multi2 = rm.open_resource('GPIB0::3::INSTR')

In [4]:
multi1.write('D3')

(4L, <StatusCode.success: 0>)

# Test

In [17]:
labjack = ljm.openS("T7", "usb", "CryoTester_ljT7")
ljm.eWriteName(labjack, 'AIN0_EF_INDEX', 3) #average
ljm.eReadName(labjack, 'AIN0')

0.007935549132525921

# tseT

In [29]:
ljm.close(labjack_ian)

NameError: name 'labjack_ian' is not defined

In [12]:
version = 25

power_start = 1 #W
resistance = 12 #ohms
voltage_start = 5. #voltage
duration = 500. #seconds
v_factor = 50./5. #factor turning 0-5V signal to the output voltage
i_factor = 24./5. #factor turning 0-5V signal to the output current
with open('log%d.txt'%version, 'w') as f_t:
    f_t.write('ctime\t power\t v_meas\t i_meas\t v_multi1\t v_multi2\t temp1\n')
    f_t.write('seconds\t Watt\t Volts\t Amps\t Volts\t Volts\t Kelvin\n')

In [11]:
ljm.eWriteName(labjack, 'AIN0_EF_CONFIG_A', 2000)
ljm.eWriteName(labjack, 'AIN0_EF_CONFIG_D', 100000)

LJMError: LJM library error code 1224 LJME_DEVICE_NOT_OPEN

voltage = 0
print voltage, 'V'    
to = time.time()
ljm.eWriteName(labjack, ps_switch, 0) #turn on the power supply
ljm.eWriteName(labjack, v_ctrl, voltage/v_factor)
time.sleep(3.)
while (time.time() - to) < 5*duration:
    time_t = time.time()
    #v_t = ljm.eReadName(labjack, v_meas) * v_factor
    v_t = float(multi2.query('F1RAZ0N4T3')) * v_factor
    i_t = ljm.eReadName(labjack, i_meas) * i_factor
    power_t = v_t*i_t
    v_multi = float(multi1.query('F1RAZ0N4T3'))
    temp1_t = lakeshore_util.ls_query('krdg? a')[0]
    line = '%.3f \t %.3f \t %.3f \t %.3f \t %.3f \t %.3f\n'%(time_t, power_t, v_t, i_t, v_multi, temp1_t)
    with open('log%d.txt'%version, 'a') as f_t:
        f_t.write(line)
ljm.eWriteName(labjack, ps_switch, 1) #Shutdown the power supply

In [20]:
for i in range(5):
    power = power_start + i*2
    voltage = np.sqrt(power*resistance)
    print power, 'W', voltage, 'V'    
    to = time.time()
    #ljm.eWriteName(labjack, ps_switch, 0) #turn on the power supply
    #ljm.eWriteName(labjack, v_ctrl, voltage/v_factor)
    ljm.eWriteName(labjack, 'AIN0_EF_CONFIG_A', 2000)
    ljm.eWriteName(labjack, 'AIN0_EF_CONFIG_D', 100000)
    #time.sleep(3.)
    while (time.time() - to) < duration:
        time_t = time.time()
        v_t = ljm.eReadName(labjack, v_meas) #rms
        v_o = ljm.eReadName(labjack, v_off)  #DC offset
        #v_t = float(multi2.query('F1RAZ0N4T3')) * v_factor
        #v_t = v_factor * (ljm.eReadName(labjack, 'AIN1') - ljm.eReadName(labjack, 'AIN0'))
        #i_t = ljm.eReadName(labjack, i_meas) * i_factor
        #power_t = v_t*i_t
        #v_multi1 = float(multi1.query('F1RAZ0N4T3'))
        
        #v_multi2 = float(multi2.query('F1RAZ0N4T3')) * v_factor
        #temp1_t = lakeshore_util.ls_query('krdg? a')[0]
        line = '%.3f \t %.8f \t %.8f\n'%(time_t, v_t, v_o)
        with open('log%d.txt'%version, 'a') as f_t:
            f_t.write(line)
#ljm.eWriteName(labjack, ps_switch, 1) #Shutdown the power supply
#ljm.close(labjack)


1 W 3.4641016151377544 V


LJMError: LJM library error code 1239 LJME_RECONNECT_FAILED

In [ ]:
for i in range(5):
    power = power_start + i*2
    voltage = np.sqrt(power*resistance)
    print power, 'W', voltage, 'V'    
    to = time.time()
    ljm.eWriteName(labjack, ps_switch, 0) #turn on the power supply
    ljm.eWriteName(labjack, v_ctrl, voltage/v_factor)
    ljm.eWriteName(labjack, 'AIN0_EF_CONFIG_A', 1250)
    ljm.eWriteName(labjack, 'AIN0_EF_CONFIG_B', 100)
    ljm.eWriteName(labjack, 'AIN0_EF_CONFIG_D', 12500)
    time.sleep(3.)
    while (time.time() - to) < duration:
        time_t = time.time()
        v_t = ljm.eReadName(labjack, v_meas) #rms
        v_o = ljm.eReadName(labjack, v_off)  #DC offset
        #v_t = float(multi2.query('F1RAZ0N4T3')) * v_factor
        #v_t = v_factor * (ljm.eReadName(labjack, 'AIN1') - ljm.eReadName(labjack, 'AIN0'))
        #i_t = ljm.eReadName(labjack, i_meas) * i_factor
        #power_t = v_t*i_t
        #v_multi1 = float(multi1.query('F1RAZ0N4T3'))
        
        #v_multi2 = float(multi2.query('F1RAZ0N4T3')) * v_factor
        #temp1_t = lakeshore_util.ls_query('krdg? a')[0]
        line = '%.3f \t %.3f \t %.8f \t %.8f\n'%(time_t, power_t, v_t, v_o)
        with open('log%d.txt'%version, 'a') as f_t:
            f_t.write(line)

voltage = 0
print voltage, 'V'    
to = time.time()
#ljm.eWriteName(labjack, ps_switch, 0) #turn on the power supply
#ljm.eWriteName(labjack, v_ctrl, voltage/v_factor)
#time.sleep(3.)
while (time.time() - to) < 5*duration:
    time_t = time.time()
    v_t = ljm.eReadName(labjack, v_meas) * v_factor
    #v_t = float(multi2.query('F1RAZ0N4T3')) * v_factor
    i_t = ljm.eReadName(labjack, i_meas) * i_factor
    power_t = v_t*i_t
    v_multi1 = float(multi1.query('F1RAZ0N4T3'))
    v_multi2 = float(multi2.query('F1RAZ0N4T3')) * v_factor
    temp1_t = lakeshore_util.ls_query('krdg? a')[0]
    line = '%.3f \t %.3f \t %.3f \t %.3f \t %.3f \t %.3f \t %.3f\n'%(time_t, power_t, v_t, i_t, v_multi1, v_multi2, temp1_t)
    with open('log%d.txt'%version, 'a') as f_t:
        f_t.write(line)
ljm.eWriteName(labjack, ps_switch, 1) #Shutdown the power supply

1 W 3.4641016151377544 V


In [11]:
ljm.eWriteName(labjack, ps_switch, 1) #Shutdown the power supply

# Original without Averaging

In [10]:
#Labjack
ps_switch = 'FIO0'
v_ctrl = 'DAC0'
v_meas = 'AIN0_EF_READ_A'
i_meas = 'AIN1_EF_READ_A'

labjack = ljm.openS("T7", "usb", "CryoTester_ljT7")
ljm.eWriteName(labjack, ps_switch, 1) #Shutdown the power supply by default
ljm.eWriteName(labjack, 'AIN_ALL_EF_INDEX', 0) #Turn off all the analog inputs
#Labjack input parameter setup
sample_rate = 200*1000
n_ch = 2
avg_time = 0.05 #s
n_sample = int(sample_rate/n_ch*avg_time)
print '%d channels, total sampling rate %d, averaging time %d s, %d sample for average.'%(n_ch, sample_rate, avg_time, n_sample)
ljm.eWriteName(labjack, 'AIN0_EF_INDEX', 3) #AIN0 average
ljm.eWriteName(labjack, 'AIN0_EF_CONFIG_A', n_sample)
ljm.eWriteName(labjack, 'AIN0_EF_CONFIG_D', sample_rate/n_ch)
ljm.eWriteName(labjack, 'AIN1_EF_INDEX', 3) #AIN1 average
ljm.eWriteName(labjack, 'AIN1_EF_CONFIG_A', n_sample)
ljm.eWriteName(labjack, 'AIN1_EF_CONFIG_D', sample_rate/n_ch)

#Multimeters
rm = visa.ResourceManager()
multi1 = rm.open_resource('GPIB0::26::INSTR')
multi2 = rm.open_resource('GPIB0::3::INSTR')

2 channels, total sampling rate 200000, averaging time 0 s, 5000 sample for average.


In [11]:
version = 1

power_start = 1 #W
resistance = 12 #ohms
duration = 200. #seconds
v_factor = 50./5. #factor turning 0-5V signal to the output voltage
i_factor = 24./5. #factor turning 0-5V signal to the output current
with open('log%d.txt'%version, 'w') as f_t:
    f_t.write('ctime\t power\t v_meas\t v_meas_no_avg\t i_meas\t v_multi1\t v_multi2\t temp1\n')
    f_t.write('seconds\t Watt\t Volts\t Volts\t Amps\t Volts\t Volts\t Kelvin\n')

In [12]:
for i in range(5):
    power = power_start + i
    voltage = np.sqrt(power * resistance)
    print power, 'W', voltage, 'V'    
    to = time.time()
    ljm.eWriteName(labjack, ps_switch, 0) #turn on the power supply
    ljm.eWriteName(labjack, v_ctrl, voltage/v_factor)
    time.sleep(3.)
    while (time.time() - to) < duration:
        time_t = time.time()
        v_t = ljm.eReadName(labjack, v_meas) * v_factor
        v_t_no_avg = ljm.eReadName(labjack, 'AIN0') * v_factor
        i_t = ljm.eReadName(labjack, i_meas) * i_factor
        power_t = v_t*i_t
        v_multi1 = float(multi1.query('F1RAZ0N4T3'))
        v_multi2 = float(multi2.query('F1RAZ0N4T3')) * v_factor
        temp1_t = lakeshore_util.ls_query('krdg? a')[0]
        line = '%.3f \t %.3f \t %.3f \t %.3f \t %.3f \t %.3f \t %.3f \t %.3f\n'%(time_t, power_t, v_t, v_t_no_avg, i_t, v_multi1, v_multi2, temp1_t)
        with open('log%d.txt'%version, 'a') as f_t:
            f_t.write(line)
power = 0 #W
voltage = np.sqrt(power * resistance)
print power, 'W', voltage, 'V'    
to = time.time()
ljm.eWriteName(labjack, ps_switch, 0) #turn on the power supply
ljm.eWriteName(labjack, v_ctrl, voltage/v_factor)
time.sleep(3.)
while (time.time() - to) < 10*duration:
    time_t = time.time()
    v_t = ljm.eReadName(labjack, v_meas) * v_factor
    v_t_no_avg = ljm.eReadName(labjack, 'AIN0') * v_factor
    i_t = ljm.eReadName(labjack, i_meas) * i_factor
    power_t = v_t*i_t
    v_multi1 = float(multi1.query('F1RAZ0N4T3'))
    v_multi2 = float(multi2.query('F1RAZ0N4T3')) * v_factor
    temp1_t = lakeshore_util.ls_query('krdg? a')[0]
    line = '%.3f \t %.3f \t %.3f \t %.3f \t %.3f \t %.3f \t %.3f \t %.3f\n'%(time_t, power_t, v_t, v_t_no_avg, i_t, v_multi1, v_multi2, temp1_t)
    with open('log%d.txt'%version, 'a') as f_t:
        f_t.write(line)
ljm.eWriteName(labjack, ps_switch, 1) #Shutdown the power supply
ljm.close(labjack)
multi1.close()
multi2.close()

1 W 3.4641016151377544 V
2 W 4.898979485566356 V


KeyboardInterrupt: 

In [5]:
ljm.eWriteName(labjack, ps_switch, 1)
ljm.close(labjack)
multi1.close()
multi2.close()